# Collision Avoidance - ResNet18 TensorRTでの自動走行

Pytorchで学習したモデルをTensorRTモデルに変換したことで高速処理が可能になりました。  
このノートブックでは、TensorRT化したモデルを使うことでカクツキを抑えてJetBotがなめらかに走行することを確認できます。  

# デバイスの準備

カメラ画像をGPUメモリに転送するために、先に定義だけしておきます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch

########################################
# TensorRTの場合、モデルはGPUを使うように実装されていますが、
# 入力データはGPUメモリに転送する必要があります。
# そのための定義をここでしておきます。
########################################
device = torch.device('cuda')

TensorRTに最適化されたモデル``best_model_trt.pth``を読み込みます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torchvision  # これはすでに読込んでいるため、省略可能です。
from torch2trt import TRTModule  # TensorRTのライブラリを利用します。

########################################
# TensorRTモデルを読み込みます。
########################################
model_trt = TRTModule()  # TensorRTモデルを読み込むための変数を定義します。
model_trt.load_state_dict(torch.load('detection_trt.pth'))  # 学習したTensorRTモデルを読み込みます。

## カメラ画像の前処理作成
モデルを読み込みましたが、まだ少し問題があります。  
学習時の入力画像フォーマットと、OpenCVのカメラ画像フォーマットは一致しません。  
これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。

1. カメラ画像をBGRフォーマットからRGBフォーマットに変換します。
> 学習時のjpeg画像はtorchvision.datasets.ImageFolderによって読み込まれます。ImageFolderはPILライブラリを利用して画像ファイルを読み込んだ後、RGBフォーマットに変換しています。このため、学習時の入力画像データはRGBフォーマットになっています。しかしカメラ画像を取得するために使っているOpenCVはデフォルトでBGRフォーマットとなるため、このまま予測すると画像の赤色と青色が入れ替わっているために精度が悪くなります。そこで、カメラ画像を学習時のフォーマットに合わせるためにRGBフォーマットに変換します。
2. HWCをCHWに変換します。
> cudnnはHWC(Height x Width x Channel)をサポートしません。そのため画像情報の並び順をHWCからCHW(Channel x Height x Width)に変換します。
3. カメラ画像を正規化します。
> トレーニング中に使ったのと同じパラメータ（平均と標準偏差）を利用してカメラ画像の各チャンネル(RGB)を正規化します。  
> OpenCVで取得したカメラ画像の1ピクセルはRGBをそれぞれint型で[0, 255]の範囲で表したものになります。  
> しかし、学習時はImageFolderを使ってjpeg画像を読込み、それをtransforms.ToTensor()を使ってTensor型に変換しています。この時、CHWへの変換と計算グラフレイヤの追加の他に、RGB値がint型の[0, 255]からfloat型の[0.0, 1.0]にスケーリングされています。学習時はこの[0.0, 1.0]の値に対してtransforms.Normalize()を行うことでRGB各値を正規化（ImageNetデータセットのRGB毎に平均を0、標準偏差が1になるようにスケーリング）しています。 
> ここではToTensor()を使わずにCHWへの変換をおこなっています。そしてImageNetと同じ範囲に各チャンネルをスケーリングするために、Normalize()に渡すパラメータに255.0を掛けています。
4. カメラ画像をGPUメモリに転送します。
> 入力データはモデルと同じデバイスに存在する必要があります。
5. 入力画像データを配列に変更します。
> 学習時はバッチサイズ分の画像を配列にして入力層に与えています。モデルの入力層は学習ために入力データを可変長の配列で受け取る構造になっています。そのため予測時は1枚の画像であっても入力画像データを配列にする必要があります。

* ImageFolderリファレンス：
  * https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder
* ImageFolder実装コード：
  * https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
* Normalizeリファレンス：
  * https://pytorch.org/docs/stable/torchvision/transforms.html
* Normalize実装コード：
  * https://github.com/pytorch/vision/blob/master/torchvision/transforms/transforms.py
* 正規化パラメータの値の理由：
  * https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
* 正規化に意味があるのかどうか：
  * https://teratail.com/questions/234027

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

########################################
# 衝突回避モデルのプリプロセッシング処理の中で使われます。
# この値はpytorch ImageNetの学習に使われた正規化のパラメータです。（ImageNetデータセットのRGB毎に平均を0、標準偏差が1になるようにスケーリングすること）
# カメラ画像はこの値でRGBを正規化することが望ましいでしょう。
########################################
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

########################################
# この正規化の定義は使っていません。
# 代わりに
# image.sub_(mean[:, None, None]).div_(std[:, None, None])
# で正規化しています。
########################################
normalize = torchvision.transforms.Normalize(mean, std)

########################################
# カメラ画像をモデル入力用データに変換します。
########################################
def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # カメラ映像はOpenCVで読み込んでいるため画像はBGRフォーマットになっています。これをRGBフォーマットに変換します。
    image = PIL.Image.fromarray(image)  # OpenCV画像データ(配列データ）をPILイメージオブジェクトに変換します。
    image = transforms.functional.to_tensor(image).to(device).half()  # 画像をTensor型に変換してfloat16型でGPUメモリに転送します。
    image.sub_(mean[:, None, None]).div_(std[:, None, None])  # ImageNetのパラメータで正規化します。
    return image[None, ...]  # バッチ配列に変換して返します。

> すばらしい、これでカメラ画像をニューラルネットワークの入力フォーマットに変換するための、pre-processing関数を定義できました。　

## カメラの動作確認
それでは、カメラを起動して表示しましょう。  
JetBotが「blocked」（旋回する）と判断している確率を表示するためのスライダーも用意します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import traitlets  # カメラ画像などのデータが更新されたときに、連動して処理を実行させるためにtraitletsライブラリを利用します。
from IPython.display import display  # ウィジェットを表示するためのdisplayライブラリを利用します。
import ipywidgets.widgets as widgets  # Jupyter標準のウィジェットを利用します。
from jetbot import Camera, bgr8_to_jpeg  # JetBot用に用意したカメラと画像変換ライブラリを利用します。

########################################
# カメラを有効化します。
# 画像はwidthとheightで指定したピクセルサイズにリサイズされます。
# fpsのデフォルトは30ですが、カメラフレーム更新に連動して推論を実行するようにコーディングしているため、
# 処理が重くなってしまいます。そのためfpsを小さく設定します。
########################################
camera = Camera.instance(width=224, height=224, fps=4)

########################################
# 画像表示用のウィジェットを用意します。
# widthとheightは表示するウィジェットの幅と高さです。
# カメラ画像サイズと一致する必要はありません。
########################################
image = widgets.Image(format='jpeg', width=224, height=224)

########################################
# 「blocked」の確率を表示するためのスライダーを用意します。
########################################
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

########################################
# 「speed」を調整するためのスライダーを用意します。
########################################
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

########################################
# traitletsライブラリを利用してカメラ画像データが更新されたときに、
# bgr8フォーマットをjpegフォーマットに変換してから
# 画像表示ウィジェットに反映するように設定します。
########################################
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

########################################
# 画像表示ウィジェットとスライダーをブラウザに表示します。
########################################
display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

モーターを制御するためにrobotインスタンスを生成します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jetbot import Robot  # JetBotを制御するためのライブラリを利用します。

########################################
# JetBotの制御用クラスをインスタンス化します。
########################################
robot = Robot()

次は、カメラの画像が更新されるたびに呼び出される関数を生成します。この関数は、下記ステップを実行します。

1. カメラ画像をPre-processingにかけてモデル入力データに変換します。
2. モデルの推論を実行します。
3. 推論結果が50%以上の確率で「blocked」の場合は、左に曲がります。それ以外の場合は前進します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch.nn.functional as F
import time

########################################
# カメラ画像が更新されたときに実行する処理を定義します。
########################################
def update(change):
    ########################################
    # この関数内で参照だけされる定義は暗黙的にグローバル定義となります。
    # しかし、この関数内で値を代入される定義は暗黙的にローカル定義となります。
    # blocked_sliderとrobotはそれ自体に値が代入されていないため、暗黙的にグローバル定義となります。
    # そのためここでのglobal宣言は省略可能です。
    ########################################
    global blocked_slider, robot
    # カメラ画像を変数xにコピーします。
    x = change['new'] 
    # カメラ画像をモデルの入力データに変換します。
    x = preprocess(x)
    # 推論を実行します。
    y = model_trt(x)
    #print(y)
    
    # softmax()関数を適用して出力ベクトルの合計が1になるように正規化します（これにより確率分布になります）
    y = F.softmax(y, dim=1)
    #print(y)
    
    # 入力データは多次元のバッチ配列になっています。出力もそれに対応しているためyは多次元配列になっています。
    # y.flatten()を呼び出すことで可能な限り不要な次元を除去します。([[blocked_rate, free_rate]]を[blocked_rate, free_rate]に変換)
    # そのうえで、「blocked」の確率となるy.flatten()[0]の値を取得します。「free」の確率を取得する場合はy.flatten()[1]になります。
    prob_blocked = float(y.flatten()[0])
    #print(prob_blocked)
    
    # 「blocked」の確率をスライダーに反映します。
    blocked_slider.value = prob_blocked
    
    # 「blocked」の確率が50%未満なら直進します。それ以外は左に旋回します。
    if prob_blocked < 0.5:
        robot.forward(speed_slider.value)  # JetBotのモーター出力をspeedスライダーの値にして前進します。
    else:
        robot.left(speed_slider.value)  # JetBotのモーター出力をspeedスライダーの値にして左に旋回します。
    
    time.sleep(0.001)  # 値がモーター制御基板のICチップに反映されるまで少し待ちます。

モデル推論からJetBotの動作までを実行する関数を作成しました。  
今度はそれをカメラ画像の更新に連動して動作させる必要があります。

JetBotでは、traitlets.HasTraitsを継承したCameraクラスを実装しているので、observe()を呼び出すだけで実現できます。

## JetBotを動かしてみよう
次のコードで``start jetbot``ボタンと``stop jetbot``ボタンを作成します。  

``start jetbot``ボタンを押すとモデルの初期化が実行されます。  
モデルの初期化が完了すると、``blocked``スライダーが動作し始めます。  
``speed``スライダーを動かすとJetBotが動作し始めます。  

``stop jetbot``ボタンを押すとJetBotが停止します。  
最初の1フレームの実行時にメモリの初期化が実行されるので、ディープラーニングではどんなモデルも最初の1フレームの処理はすこし時間がかかります。

In [ ]:
########################################
# スタートボタンとストップボタンを作成します。
########################################
model_start_button = widgets.Button(description='start jetbot')
model_stop_button = widgets.Button(description='stop jetbot')

########################################
# スタートボタンがクリックされた時に呼び出す関数を定義します。
########################################
def start_model(c):
    update({'new': camera.value})  # update()関数を1回呼び出して初期化します。
    camera.observe(update, names='value')  # Cameraクラスのtraitlets.Any()型のvalue変数(カメラ画像データ)が更新されたときに指定した関数を呼び出します。
model_start_button.on_click(start_model)  # startボタンがクリックされた時に指定した関数を呼び出します。

########################################
# ストップボタンがクリックされた時に呼び出す関数を定義します。
########################################    
def stop_model(c):
    camera.unobserve(update, names='value')  # カメラ画像が更新されたときにupdate()関数を呼び出していた接続を解除します。
    time.sleep(1)  # フレームの処理の完了を待つためのスリープを追加します。
    robot.stop()  # モーターを停止します。
model_stop_button.on_click(stop_model)  # stopボタンがクリックされた時に指定した関数を呼び出します。

########################################
# ウィジェットの表示レイアウトを定義します。
########################################
model_widget = widgets.VBox([
    widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]),
    widgets.HBox([model_start_button, model_stop_button])
])

########################################
# ウィジェットを表示します。
########################################
display(model_widget)

# カメラの停止
最後に、他のノートブックでカメラを使うために、このノートブックで使ったカメラを停止しておきます。

In [ ]:
camera_link.unlink()  # ブラウザへのストリーミングを停止します。（JetBot本体でのカメラは動作し続けます。）
camera.stop()  # カメラを停止します。

### 結論
このライブデモは以上です。うまくいけばあなたのJetBotは賢く衝突を避けていることでしょう。

collision avoidanceが上手く行かない場合、正しく走行できるように失敗しやすい場所でさらにデータを追加してください。  
このようにうまくいかない場所を中心にデータを収集すれば、JetBotはさらによく動作するはずです。